# Master File Outlining Profile Construction Code

# 0. Python imports

In [ ]:
import pandas as pd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import interpolate
from metrCalculator import calc_theta
from metrCalculator import calc_mix_ratio
from matplotlib.backends.backend_pdf import PdfPages

# 1. Calculate Averaged Profile from Surface to Cloud Top

## 1a. Set user defined variables

<strong>filepath</strong> = path to flight data<br>
<strong>filepath_ice</strong> = path to 2DS data<br>
<strong>filepath_aero</strong> = path to SMPS data<br>
<strong>spath</strong> = path to save output files to<br>
<strong>sname</strong> = identifier for what profile is being name. will be used in the name of output files<br>

<strong>start</strong> = first index of main flight data to be used for profile from surface to cloud top<br>
<strong>stop</strong> = last index of main flight data to be used for profile from surface to cloud top<br>

<strong>start_2</strong> = first index of main flight data to be used for profile from cloud top to flight path top<br>
<strong>stop_2</strong> = last index of main flight data to be used for profile from cloud top to flight path top<br>

<strong>interval</strong> = altitude interval to be averaged over<br>
<strong>interval_input</strong>   = altitude interval for input profile<br>

<strong>heights</strong> = array of altitude levels to be averaged over (user sets max height)<br>
<strong>z_start</strong> = height in m to start above cloud profile at<br>
<strong>flightPathTop</strong> = height of the top of the flight path<br>
<strong>heights_2</strong> = array of altitude levels from cloud top to flight path top<br>

<strong>rho_aero</strong> = density of aerosol in kg/m^3<br>
<strong>rho_air</strong> = density of air in kg/m^3<br>

<strong>aerosol_start</strong> = first index of SMPS data to be included<br>
<strong>aerosol_stop</strong> = last index of SMPS data to be included<br>


<strong>SEE PART 12 FOR MCF USER DEFINED VARIABLES</strong>

In [ ]:
filepath = '/flight219/core_masin_20151127_r002_flight219_1hz[1].nc'
filepath_ice = '/flight219/UMAN_2DS_20151127_r1_Flight219[1].nc'
filepath_aero = '/DataFiles/MAC_smps_dn_cm3.xlsx'
spath = '/Profiles/F219/'
### save name ###
sname = 'flight219'
###           ###

start = 3167 #~start of cloud pass
stop = 15187 # end of flight

start_2 = 0 # start of flight
stop_2 = 3167 # start of cloud pass

interval = 10 #[m]
interval_input = 50 #[m]

# Make an array of the hieght levels
heights = np.arange(0,4020,interval) #Surface to domain top
z_start = 1070 # ~cloud pass top
flightPathTop = 1550 # ~flight path top
heights_2 = np.arange(z_start,flightPathTop,interval) #~cloud pass top to ~flight path top

cBase = 370 # cloud base height[m] Used in step 6 to set random noise (from table in paper)

rho_aero = 2.170 #kg/m3
rho_air = 1.225 #kg/m3

aerosol_start = 153 #Selects the correct date from the campaign smps data file
aerosol_stop = 173 #Selects the correct date from the campaign smps data file


## 1b. Read in data, select variables, and save in DataFrame

In [ ]:
data = Dataset(filepath)

# Make an nan array to be replaced later
nans = np.arange(0,len(data['Time'][start:stop]),1)
nans = np.full_like(nans,np.nan,dtype=np.double)

selectedData = pd.DataFrame(data={'time[s]':data.variables['Time'][start:stop],'T_DI[K]':data.variables['TAT_DI_R'][start:stop],'T_ND[K]':data.variables['TAT_ND_R'][start:stop],'Td[K]':data.variables['TDEW_BUCK'][start:stop],'Alt[m]':data.variables['ALT_OXTS'][start:stop],'RH':data.variables['HUM_ROSE'][start:stop],'Lat':data.variables['LAT_OXTS'][start:stop],'Lon':data.variables['LON_OXTS'][start:stop],'P[hPa]':data.variables['PS_AIR'][start:stop],'u[m/s]':data.variables['U_OXTS'][start:stop],'v[m/s]':data.variables['V_OXTS'][start:stop],'w[m/s]':data.variables['W_OXTS'][start:stop],'NC_All':nans[:],'NC_HI':nans[:],'NC_MI':nans[:],'NC_LI':nans[:],'NC_S':nans[:],'NC_Accum':nans[:],'Mass_Accum':nans[:],'NC_Aitk':nans[:],'Mass_Aitk':nans[:],'NC_Total':nans[:],'Mass_Total':nans[:],'RandomNoise':nans[:]})

## 1c. Insert ice data

In [ ]:
iceData = Dataset(filepath_ice)

iceTime = pd.DataFrame(data={'Time':iceData['Time_mid'][:]-0.5})
time_start = min(selectedData['time[s]'][:]*1.0)
start_ice = iceTime[iceTime['Time'] == time_start].index.tolist()
start_ice = start_ice[0]
stop_ice = len(iceTime[:])

iceSet = pd.DataFrame(data={'Time':iceData['Time_mid'][start_ice:stop_ice]-0.5,'NC_All':iceData['NC_All'][start_ice:stop_ice],'NC_HI':iceData['NC_HI'][start_ice:stop_ice],'NC_MI':iceData['NC_MI'][start_ice:stop_ice],'NC_LI':iceData['NC_LI'][start_ice:stop_ice],'NC_S':iceData['NC_S'][start_ice:stop_ice]})


#Loop through main data and if the time matches the time in ice data then save the location values
#Check for fill values (-9999) and change any to 0.0
i = 0
j = 0
for i in range(len(selectedData)): 
    if selectedData['time[s]'][i]*1.0 == iceSet['Time'][j]:
        if iceSet.at[j,'NC_All'] == -9999:
            selectedData.at[i,'NC_All'] = 0.0
        else:
             selectedData.at[i,'NC_All'] = iceSet.at[j,'NC_All']
                
        if iceSet.at[j,'NC_HI']  == -9999:
            selectedData.at[i,'NC_HI'] = 0.0
        else:
            selectedData.at[i,'NC_HI'] = iceSet.at[j,'NC_HI']
            
        if iceSet.at[j,'NC_MI']  == -9999:
            selectedData.at[i,'NC_MI'] = 0.0
        else:
            selectedData.at[i,'NC_MI'] = iceSet.at[j,'NC_MI']
            
        if iceSet.at[j,'NC_LI']  == -9999:
            selectedData.at[i,'NC_LI'] = 0.0
        else:
            selectedData.at[i,'NC_LI'] = iceSet.at[j,'NC_LI']
    
        if iceSet.at[j,'NC_S']  == -9999:
            selectedData.at[i,'NC_S'] = 0.0
        else:
            selectedData.at[i,'NC_S'] = iceSet.at[j,'NC_S']
            
        j += 1
        

## 1d. Sort data by altitude and reset indexing

In [ ]:
selectedData = selectedData.sort_values('Alt[m]')
selectedData = selectedData.reset_index()

selectedData.to_excel(spath + sname + '_selectedData.xlsx')

## 1e. Make arrays to be used in average calcuations

In [ ]:
# Make an empty (zeros) array of the same length as hieghts
zeros = np.zeros(len(heights))

# Make an empty dataframe to store the averaged data in
avg = pd.DataFrame(data={'z[m]':heights[:],'T_DI[K]':zeros[:],'T_ND[K]':zeros[:],'RH':zeros[:],'theta_DI[K]':zeros[:],'theta_ND[K]':zeros[:],'mix_ratio_DI[kg/kg]':zeros[:],'mix_ratio_ND[kg/kg]':zeros[:],'P[hPa]':zeros[:],'u[m/s]':zeros[:],'v[m/s]':zeros[:],'w[m/s]':zeros[:],'NC_All':zeros[:],'NC_HI':zeros[:],'NC_MI':zeros[:],'NC_LI':zeros[:],'NC_S':zeros[:],'NC_Accum':zeros[:],'Mass_Accum':zeros[:],'NC_Aitk':zeros[:],'Mass_Aitk':zeros[:],'NC_Total':zeros[:],'Mass_Total':zeros[:],'RandomNoise':zeros[:]})


## 1f. Calculate the averaged profile and save to excel spreadsheet

In [ ]:
#Loop through z
for z in heights:
    sumTDI = 0
    countTDI = 0
    sumTND = 0
    countTND = 0
    sumRH = 0
    countRH = 0
    sumLat = 0
    countLat = 0
    sumLon = 0
    countLon = 0
    sumP = 0
    countP = 0
    sumU = 0
    countU = 0
    sumV = 0
    countV = 0
    sumW = 0
    countW = 0
    sumAI = 0
    countAI = 0
    sumHI = 0
    countHI = 0
    sumMI = 0
    countMI = 0
    sumLI = 0
    countLI = 0
    sumS = 0
    countS = 0

    print(z)
    #print(len(selectedData))
    selectedData = selectedData.reset_index(drop=True)

    # loop through data rows
    for i in range(len(selectedData)):
        # Check if the current altitude is in the current height interval
        # If it is then continue to the average calculations, if not do nothing
        if selectedData['Alt[m]'][i] > z and selectedData['Alt[m]'][i] < z+interval:
            # Check that the current value is not NaN. If NaN do nothing.
            # If there is a value then add it to the running sum and add 1 to count.
            if math.isnan(selectedData['T_DI[K]'][i]) == False:
                sumTDI = sumTDI + selectedData['T_DI[K]'][i]
                countTDI += 1
            if math.isnan(selectedData['T_ND[K]'][i]) == False:
                sumTND = sumTND + selectedData['T_ND[K]'][i]
                countTND += 1
            if math.isnan(selectedData['RH'][i]) == False:
                sumRH = sumRH + selectedData['RH'][i]
                countRH += 1
            if math.isnan(selectedData['Lat'][i]) == False:
                sumLat = sumLat + selectedData['Lat'][i]
                countLat += 1
            if math.isnan(selectedData['Lon'][i]) == False:
                sumLon = sumLon + selectedData['Lon'][i]
                countLon += 1
            if math.isnan(selectedData['P[hPa]'][i]) == False:
                sumP = sumP + selectedData['P[hPa]'][i]
                countP += 1
            if math.isnan(selectedData['u[m/s]'][i]) == False:
                sumU = sumU + selectedData['u[m/s]'][i]
                countU += 1
            if math.isnan(selectedData['v[m/s]'][i]) == False:
                sumV = sumV + selectedData['v[m/s]'][i]
                countV += 1
            if math.isnan(selectedData['w[m/s]'][i]) == False:
                sumW = sumW + selectedData['w[m/s]'][i]
                countW += 1
            if math.isnan(selectedData['NC_All'][i]) == False:
                sumAI = sumAI + selectedData['NC_All'][i]
                countAI += 1
            if math.isnan(selectedData['NC_HI'][i]) == False:
                sumHI = sumHI + selectedData['NC_HI'][i]
                countHI += 1
            if math.isnan(selectedData['NC_MI'][i]) == False:
                sumMI = sumMI + selectedData['NC_MI'][i]
                countMI += 1
            if math.isnan(selectedData['NC_LI'][i]) == False:
                sumLI = sumLI + selectedData['NC_LI'][i]
                countAI += 1
            if math.isnan(selectedData['NC_S'][i]) == False:
                sumS = sumS + selectedData['NC_S'][i]
                countS += 1

            # Delete the used row
            selectedData = selectedData.drop(i,axis=0)

        # Check that there were values in the interval. If there were then
        # calculate the average. If not set average to NaN.
        if countTDI > 0:
            avg.at[z/interval,'T_DI[K]'] = sumTDI/countTDI
        else:
            avg.at[z/interval,'T_DI[K]'] = 'NaN'
        if countTND > 0:
            avg.at[z/interval,'T_ND[K]'] = sumTND/countTND
        else:
            avg.at[z/interval,'T_ND[K]'] = 'NaN'
        if countRH > 0:
            avg.at[z/interval,'RH'] = sumRH/countRH
        else:
            avg.at[z/interval,'RH'] = 'NaN'
        if countLat > 0:
            avg.at[z/interval,'Lat'] = sumLat/countLat
        else:
            avg.at[z/interval,'Lat'] = 'NaN'
        if countLon > 0:
            avg.at[z/interval,'Lon'] = sumLon/countLon
        else:
            avg.at[z/interval,'Lon'] = 'NaN'
        if countP > 0:
            avg.at[z/interval,'P[hPa]'] = sumP/countP
        else:
            avg.at[z/interval,'P[hPa]'] = 'NaN'
        if countU > 0:
            avg.at[z/interval,'u[m/s]'] = sumU/countU
        else:
            avg.at[z/interval,'u[m/s]'] = 'NaN'
        if countV > 0:
            avg.at[z/interval,'v[m/s]'] = sumV/countV
        else:
            avg.at[z/interval,'v[m/s]'] = 'NaN'
        if countW > 0:
            avg.at[z/interval,'w[m/s]'] = sumW/countW
        else:
            avg.at[z/interval,'w[m/s]'] = 'NaN'
        if countAI > 0:
            avg.at[z/interval,'NC_All'] = sumAI/countAI
        else:
            avg.at[z/interval,'NC_All'] = 'NaN'
        if countHI > 0:
            avg.at[z/interval,'NC_HI'] = sumHI/countHI
        else:
            avg.at[z/interval,'NC_HI'] = 'NaN'
        if countMI > 0:
            avg.at[z/interval,'NC_MI'] = sumMI/countMI
        else:
            avg.at[z/interval,'NC_MI'] = 'NaN'
        if countLI > 0:
            avg.at[z/interval,'NC_LI'] = sumLI/countLI
        else:
            avg.at[z/interval,'NC_LI'] = 'NaN'
        if countS > 0:
            avg.at[z/interval,'NC_S'] = sumS/countS
        else:
            avg.at[z/interval,'NC_S'] = 'NaN'


avg.to_excel(spath + sname + '_avg_1.xlsx')


# 2. Calculate Averaged Profile from Cloud Top to Flight Path Top

## 2a. Read in data, select variables, and save in DataFrame


In [ ]:
data = Dataset(filepath)

# Make an nan array to be replaced later
nans = np.arange(0,len(data['Time'][start_2:stop_2]),1)
nans = np.full_like(nans,np.nan,dtype=np.double)

selectedData = pd.DataFrame(data={'time[s]':data.variables['Time'][start_2:stop_2],'T_DI[K]':data.variables['TAT_DI_R'][start_2:stop_2],'T_ND[K]':data.variables['TAT_ND_R'][start_2:stop_2],'Td[K]':data.variables['TDEW_BUCK'][start_2:stop_2],'Alt[m]':data.variables['ALT_OXTS'][start_2:stop_2],'RH':data.variables['HUM_ROSE'][start_2:stop_2],'Lat':data.variables['LAT_OXTS'][start_2:stop_2],'Lon':data.variables['LON_OXTS'][start_2:stop_2],'P[hPa]':data.variables['PS_AIR'][start_2:stop_2],'u[m/s]':data.variables['U_OXTS'][start_2:stop_2],'v[m/s]':data.variables['V_OXTS'][start_2:stop_2],'w[m/s]':data.variables['W_OXTS'][start_2:stop_2],'NC_All':nans[:],'NC_HI':nans[:],'NC_MI':nans[:],'NC_LI':nans[:],'NC_S':nans[:],'NC_Accum':nans[:],'Mass_Accum':nans[:],'NC_Aik':nans[:],'Mass_Aik':nans[:],'NC_Total':nans[:],'Mass_Total':nans[:],'RandomNoise':nans[:]})

## 2b. Insert ice data

In [ ]:
iceData = Dataset(filepath_ice)

iceTime = pd.DataFrame(data={'Time':iceData['Time_mid'][:]-0.5})
time_start = min(selectedData['time[s]'][:]*1.0)

if time_start < min(iceTime['Time'][:]):
    start_ice = iceTime[iceTime['Time'] == min(iceTime['Time'][:])].index.tolist()
else:  
    start_ice = iceTime[iceTime['Time'] == time_start].index.tolist()
    
start_ice = start_ice[0]
stop_ice = len(iceTime[:])

iceSet = pd.DataFrame(data={'Time':iceData['Time_mid'][start_ice:stop_ice]-0.5,'NC_All':iceData['NC_All'][start_ice:stop_ice],'NC_HI':iceData['NC_HI'][start_ice:stop_ice],'NC_MI':iceData['NC_MI'][start_ice:stop_ice],'NC_LI':iceData['NC_LI'][start_ice:stop_ice],'NC_S':iceData['NC_S'][start_ice:stop_ice]})


#Loop through main data and if the time matches the time in ice data then save the location values
i = 0
j = 0
for i in range(len(selectedData)): 
    if selectedData['time[s]'][i]*1.0 == iceSet['Time'][j]:
        if iceSet.at[j,'NC_All'] == -9999:
            selectedData.at[i,'NC_All'] = 0.0
        else:
             selectedData.at[i,'NC_All'] = iceSet.at[j,'NC_All']
                
        if iceSet.at[j,'NC_HI']  == -9999:
            selectedData.at[i,'NC_HI'] = 0.0
        else:
            selectedData.at[i,'NC_HI'] = iceSet.at[j,'NC_HI']
            
        if iceSet.at[j,'NC_MI']  == -9999:
            selectedData.at[i,'NC_MI'] = 0.0
        else:
            selectedData.at[i,'NC_MI'] = iceSet.at[j,'NC_MI']
            
        if iceSet.at[j,'NC_LI']  == -9999:
            selectedData.at[i,'NC_LI'] = 0.0
        else:
            selectedData.at[i,'NC_LI'] = iceSet.at[j,'NC_LI']
    
        if iceSet.at[j,'NC_S']  == -9999:
            selectedData.at[i,'NC_S'] = 0.0
        else:
            selectedData.at[i,'NC_S'] = iceSet.at[j,'NC_S']
            
        j += 1
        

## 2c. Sort data by altitude and reset indexing

In [ ]:
selectedData = selectedData.sort_values('Alt[m]')
selectedData = selectedData.reset_index()

selectedData.to_excel(spath + sname + '_selectedData_2.xlsx')

## 2d. Make arrays to be used in average calcuations

In [ ]:
# Make an empty (zeros) array of the same length as hieghts
zeros = np.zeros(len(heights_2))

# Make an empty dataframe to store the averaged data in
avg_2 = pd.DataFrame(data={'z[m]':heights_2[:],'T_DI[K]':zeros[:],'T_ND[K]':zeros[:],'RH':zeros[:],'theta_DI[K]':zeros[:],'theta_ND[K]':zeros[:],'mix_ratio_DI[kg/kg]':zeros[:],'mix_ratio_ND[kg/kg]':zeros[:],'P[hPa]':zeros[:],'u[m/s]':zeros[:],'v[m/s]':zeros[:],'w[m/s]':zeros[:],'NC_All':zeros[:],'NC_HI':zeros[:],'NC_MI':zeros[:],'NC_LI':zeros[:],'NC_S':zeros[:],'NC_Accum':zeros[:],'Mass_Accum':zeros[:],'NC_Aik':zeros[:],'Mass_Aik':zeros[:],'NC_Total':zeros[:],'Mass_Total':zeros[:],'RandomNoise':zeros[:]})


## 2e. Calculate the averaged profile and save to excel spreadsheet

In [ ]:
#Loop through z
for z in heights_2:
    sumTDI = 0
    countTDI = 0
    sumTND = 0
    countTND = 0
    sumRH = 0
    countRH = 0
    sumLat = 0
    countLat = 0
    sumLon = 0
    countLon = 0
    sumP = 0
    countP = 0
    sumU = 0
    countU = 0
    sumV = 0
    countV = 0
    sumW = 0
    countW = 0
    sumAI = 0
    countAI =0
    sumHI = 0
    countHI = 0
    sumMI = 0
    countMI = 0
    sumLI = 0
    countLI = 0
    sumS = 0
    countS = 0

    print(z)
    #print(len(selectedData))
    selectedData = selectedData.reset_index(drop=True)

    # loop through data rows
    for i in range(len(selectedData)):
        # Check if the current altitude is in the current height interval
        # If it is then continue to the average calculations, if not do nothing
        if selectedData['Alt[m]'][i] > z and selectedData['Alt[m]'][i] < z+interval:
            # Check that the current value is not NaN. If NaN do nothing.
            # If there is a value then add it to the running sum and add 1 to count.
            if math.isnan(selectedData['T_DI[K]'][i]) == False:
                sumTDI = sumTDI + selectedData['T_DI[K]'][i]
                countTDI += 1
            if math.isnan(selectedData['T_ND[K]'][i]) == False:
                sumTND = sumTND + selectedData['T_ND[K]'][i]
                countTND += 1
            if math.isnan(selectedData['RH'][i]) == False:
                sumRH = sumRH + selectedData['RH'][i]
                countRH += 1
            if math.isnan(selectedData['Lat'][i]) == False:
                sumLat = sumLat + selectedData['Lat'][i]
                countLat += 1
            if math.isnan(selectedData['Lon'][i]) == False:
                sumLon = sumLon + selectedData['Lon'][i]
                countLon += 1
            if math.isnan(selectedData['P[hPa]'][i]) == False:
                sumP = sumP + selectedData['P[hPa]'][i]
                countP += 1
            if math.isnan(selectedData['u[m/s]'][i]) == False:
                sumU = sumU + selectedData['u[m/s]'][i]
                countU += 1
            if math.isnan(selectedData['v[m/s]'][i]) == False:
                sumV = sumV + selectedData['v[m/s]'][i]
                countV += 1
            if math.isnan(selectedData['w[m/s]'][i]) == False:
                sumW = sumW + selectedData['w[m/s]'][i]
                countW += 1
            if math.isnan(selectedData['NC_All'][i]) == False:
                sumAI = sumAI + selectedData['NC_All'][i]
                countAI += 1
            if math.isnan(selectedData['NC_HI'][i]) == False:
                sumHI = sumHI + selectedData['NC_HI'][i]
                countHI += 1
            if math.isnan(selectedData['NC_MI'][i]) == False:
                sumMI = sumMI + selectedData['NC_MI'][i]
                countMI += 1
            if math.isnan(selectedData['NC_LI'][i]) == False:
                sumLI = sumLI + selectedData['NC_LI'][i]
                countAI += 1
            if math.isnan(selectedData['NC_S'][i]) == False:
                sumS = sumS + selectedData['NC_S'][i]
                countS += 1

            # Delete the used row
            selectedData = selectedData.drop(i,axis=0)

        # Check that there were values in the interval. If there were then
        # calculate the average. If not set average to NaN.
        tempz = z-z_start
        
        if countTDI > 0:
            avg_2.at[(z-z_start)/interval,'T_DI[K]'] = sumTDI/countTDI
        else:
            avg_2.at[(z-z_start)/interval,'T_DI[K]'] = 'NaN'
        if countTND > 0:
            avg_2.at[(z-z_start)/interval,'T_ND[K]'] = sumTND/countTND
        else:
            avg_2.at[(z-z_start)/interval,'T_ND[K]'] = 'NaN'
        if countRH > 0:
            avg_2.at[(z-z_start)/interval,'RH'] = sumRH/countRH
        else:
            avg_2.at[(z-z_start)/interval,'RH'] = 'NaN'
        if countLat > 0:
            avg_2.at[(z-z_start)/interval,'Lat'] = sumLat/countLat
        else:
            avg_2.at[(z-z_start)/interval,'Lat'] = 'NaN'
        if countLon > 0:
            avg_2.at[(z-z_start)/interval,'Lon'] = sumLon/countLon
        else:
            avg_2.at[(z-z_start)/interval,'Lon'] = 'NaN'
        if countP > 0:
            avg_2.at[(z-z_start)/interval,'P[hPa]'] = sumP/countP
        else:
            avg_2.at[(z-z_start)/interval,'P[hPa]'] = 'NaN'
        if countU > 0:
            avg_2.at[(z-z_start)/interval,'u[m/s]'] = sumU/countU
        else:
            avg_2.at[(z-z_start)/interval,'u[m/s]'] = 'NaN'
        if countV > 0:
            avg_2.at[(z-z_start)/interval,'v[m/s]'] = sumV/countV
        else:
            avg_2.at[(z-z_start)/interval,'v[m/s]'] = 'NaN'
        if countW > 0:
            avg_2.at[(z-z_start)/interval,'w[m/s]'] = sumW/countW
        else:
            avg_2.at[(z-z_start)/interval,'w[m/s]'] = 'NaN'
        if countAI > 0:
            avg_2.at[(z-z_start)/interval,'NC_All'] = sumAI/countAI
        else:
            avg_2.at[(z-z_start)/interval,'NC_All'] = 'NaN'
        if countHI > 0:
            avg_2.at[(z-z_start)/interval,'NC_HI'] = sumHI/countHI
        else:
            avg_2.at[(z-z_start)/interval,'NC_HI'] = 'NaN'
        if countMI > 0:
            avg_2.at[(z-z_start)/interval,'NC_MI'] = sumMI/countMI
        else:
            avg_2.at[(z-z_start)/interval,'NC_MI'] = 'NaN'
        if countLI > 0:
            avg_2.at[(z-z_start)/interval,'NC_LI'] = sumLI/countLI
        else:
            avg_2.at[(z-z_start)/interval,'NC_LI'] = 'NaN'
        if countS > 0:
            avg_2.at[(z-z_start)/interval,'NC_S'] = sumS/countS
        else:
            avg_2.at[(z-z_start)/interval,'NC_S'] = 'NaN'

avg_2.to_excel(spath + sname + '_avg_2.xlsx')


# 3. Merge the results from part 2. into the profile from part 1. and save into excel spreadsheet.

In [ ]:
j = 0
for i in heights_2[1:len(heights)]/10:
    i = int(i)
    avg.at[i,'T_DI[K]']=avg_2['T_DI[K]'][j]
    avg.at[i,'T_ND[K]']=avg_2['T_ND[K]'][j]
    avg.at[i,'RH']=avg_2['RH'][j]
    avg.at[i,'Lat']=avg_2['Lat'][j]
    avg.at[i,'Lon']=avg_2['Lon'][j]
    avg.at[i,'P[hPa]']=avg_2['P[hPa]'][j]
    avg.at[i,'u[m/s]']=avg_2['u[m/s]'][j]
    avg.at[i,'v[m/s]']=avg_2['v[m/s]'][j]
    avg.at[i,'w[m/s]']=avg_2['w[m/s]'][j]
    avg.at[i,'NC_All']=avg_2['NC_All'][j]
    avg.at[i,'NC_HI'] =avg_2['NC_HI'][j]
    avg.at[i,'NC_MI'] =avg_2['NC_MI'][j]
    avg.at[i,'NC_LI'] =avg_2['NC_LI'][j]
    avg.at[i,'NC_S'] =avg_2['NC_S'][j]
    j+=1
    
avg.to_excel(spath + sname + '_avg_3.xlsx')

# 4. Extrapolate the data at the top of the flight data to make the profile go up to 4000 m.

In [ ]:
avg_4 = avg[:]
variables = ['T_DI[K]','T_ND[K]','P[hPa]']
variables_2 = ['RH','Lat','Lon','u[m/s]','v[m/s]','w[m/s]']

for var in variables:
    x = avg_2[var][len(avg_2)-8:len(avg_2)-2]
    y = avg_2['z[m]'][len(avg_2)-8:len(avg_2)-2]

    f = interpolate.interp1d(y,x,fill_value="extrapolate",kind="linear")
    ynew = np.arange(flightPathTop,4030,10)
    xnew = f(ynew)

    j = 0
    for i in range(int(flightPathTop/10),403,1):
        avg_4.loc[i,var] = xnew[j]
        j += 1


for var in variables_2:
    x = avg_2[var][len(avg_2)-8:len(avg_2)-2]
    y = avg_2['z[m]'][len(avg_2)-8:len(avg_2)-2]

    f = interpolate.interp1d(y,x,fill_value="extrapolate",kind="previous")
    ynew = np.arange(flightPathTop,4030,10)
    xnew = f(ynew)

    j = 0
    for i in range(int(flightPathTop/10),403,1):
        avg_4.loc[i,var] = xnew[j]
        j += 1


avg_4.to_excel(spath + sname + '_avg_4.xlsx')


# 5. Calculate potential temperature and vapour mixing ratio

In [ ]:
for i in range(len(avg)):
    # if any of the values are NaN then skip the row, otherwise run the calculation
    if avg_4.at[i,'theta_DI[K]'] == 'NaN' or avg_4.at[i,'P[hPa]'] == 'NaN':
        print('NaN at row: ' + str(i))
        avg_4.at[i,'theta_DI[K]'] = 'NaN'
    else:
        avg_4.at[i,'theta_DI[K]'] = calc_theta(avg_4.at[i,'T_DI[K]'],avg_4.at[i,'P[hPa]'],1000)
    
    if avg_4.at[i,'theta_ND[K]'] == 'NaN' or avg_4.at[i,'P[hPa]'] == 'NaN':
        print('NaN at row: ' + str(i))
        avg_4.at[i,'theta_ND[K]'] = 'NaN'
    else:
        avg_4.at[i,'theta_ND[K]'] = calc_theta(avg_4.at[i,'T_ND[K]'],avg_4.at[i,'P[hPa]'],1000)
    
    if avg_4.at[i,'mix_ratio_DI[kg/kg]'] == 'NaN' or avg_4.at[i,'P[hPa]'] == 'NaN':
        print('NaN at row: ' + str(i))
        avg_4.at[i,'mix_ratio_DI[kg/kg]'] = 'NaN'
    else:
        avg_4.at[i,'mix_ratio_DI[kg/kg]'] = calc_mix_ratio(avg_4.at[i,'P[hPa]'],avg_4.at[i,'T_DI[K]'],avg.at[i,'RH'])
    
    if avg_4.at[i,'mix_ratio_ND[kg/kg]'] == 'NaN' or avg_4.at[i,'P[hPa]'] == 'NaN':
        print('NaN at row: ' + str(i))
        avg_4.at[i,'mix_ratio_ND[kg/kg]'] = 'NaN'
    else:
        avg_4.at[i,'mix_ratio_ND[kg/kg]'] = calc_mix_ratio(avg_4.at[i,'P[hPa]'],avg_4.at[i,'T_ND[K]'],avg.at[i,'RH'])
    
    
avg_4.to_excel(spath + sname + '_avg_5.xlsx')


# 6 Set random noise to 0.1 below cloud base and 0.0001 above cloud base

In [ ]:
cBaseIndex = np.where(avg_4['z[m]'][:] == cBase)

for i in range(0,cBaseIndex[0][0],1):
    avg_4.at[i,'RandomNoise'] = 0.1

for i in range(cBaseIndex[0][0],len(heights)):
    avg_4.at[i,'RandomNoise'] = 0.0001

# 7. Calculate aerosol data and input into dataframe


## 7a. Calcuate aerosol number concentration

In [ ]:
#Loads in data and gets the dataframe into a nice format
data = pd.read_excel(filepath_aero)
data = data[aerosol_start:aerosol_stop]
binSizes = data.columns.values.tolist() #saves bin sizes for reference later
data = data.reset_index()
data = data.drop(columns='index')
data = data.T
data = data.rename({'Unnamed: 0':'Date','Unnamed: 1':'time',20.4250952021526:0, 22.4325910999881:1, 24.6437117917855:2, 27.0809844387799:3, 29.7694227451051:4, 32.7357551943633:5, 36.0153628336391:6, 39.6414443158088:7, 43.657769753255:8, 48.1074387452588:9, 53.0463834603934:10, 58.5367022693272:11, 64.6494708948485:12, 71.4701788306466:13, 79.0960792172577:14, 87.6397387949133:15, 97.2414070232693:16, 108.05511526259:17, 120.278143540756:18, 134.136463370406:19, 149.913582565337:20, 167.943965022015:21, 188.639754038945:22, 212.483927696124:23, 240.09271275061:24, 272.20857095331:25, 309.605498652495:26}, axis='index')

# Aitken mode
sum_Aitk = np.zeros((aerosol_stop-aerosol_start))

#Calculate running sum of Aitken particles. (Ignore negative values)
for c in range(0,(aerosol_stop-aerosol_start),1):
    for i in range(0,17,1):
        if data[c][i] >= 0:
            sum_Aitk[c] = sum_Aitk[c] + data[c][i]

Aitk_Total_avg = 0 #Total number of Aitken particles per cm^3
for i in range(len(sum_Aitk)):
    Aitk_Total_avg = Aitk_Total_avg + sum_Aitk[i]

Aitk_Total_avg = Aitk_Total_avg / (aerosol_stop - aerosol_start)
Aitk_Total_avg = Aitk_Total_avg * 1000000.0 #Total number of Aitken particles per m^3
Aitk_Total_avg = Aitk_Total_avg / rho_air #Convert to per kg


# Accumulation mode
sum_Accu = np.zeros((aerosol_stop-aerosol_start))

#Calculate running sum of Aitken particles. (Ignore negative values)
for c in range(0,(aerosol_stop-aerosol_start),1):
    for i in range(17,26,1):
        if data[c][i] >= 0:
            sum_Accu[c] = sum_Accu[c] + data[c][i]

Accu_Total_avg = 0 #Total number of Accumulation particles per cm^3
for i in range(len(sum_Accu)):
    Accu_Total_avg = Accu_Total_avg + sum_Accu[i]

Accu_Total_avg = Accu_Total_avg / (aerosol_stop - aerosol_start)
Accu_Total_avg = Accu_Total_avg * 1000000.0 #Total number of Accumulation particles per m^3
Accu_Total_avg = Accu_Total_avg / rho_air #Convert to per kg

aero_Total_avg = Aitk_Total_avg + Accu_Total_avg

## 7b. Input results into dataframe (note: assumes constant distribution)

In [ ]:
for i in range(len(avg_4)):
    avg_4.loc[i,'NC_Aitk'] = Aitk_Total_avg
    avg_4.loc[i,'NC_Accu'] = Accu_Total_avg
    avg_4.loc[i,'NC_Total'] = aero_Total_avg
    

## 7c. Calculate the total mass in each size bin

In [ ]:
#Calculate the total number of particles in each size bin (for all modes)
sizeBins = np.zeros(len(binSizes)-2)
for i in range(len(sizeBins)):
    runningSum = 0
    count = 0
    for j in range(len(data.columns)):
        if data[j][i] >= 0:
            runningSum = runningSum + data[j][i]
            count += 1
    
    if count > 0:
        sizeBins[i] = runningSum / count
    
#Conver bin diameters from nm to m
D = np.zeros(len(binSizes)-2)
for i in range(len(D)):
    D[i] = float(binSizes[i+2]) * (10**(-9))

#Calculate mass in each size bin (for all modes)
for i in range(len(sizeBins)):
    sizeBins[i] = (D[i] ** 3) * (math.pi/6) * rho_aero * sizeBins[i]
    

## 7d. Calculate the mass mixing ratio for Akitken and Accumulation modes

In [ ]:
#Calculate the total mass (Aitken mode)
totalMass = 0
for i in range(17):
    totalMass = totalMass + sizeBins[i]

#Convert to kg/kg
massMixingRatio_Aitken = (totalMass * (10**6)) / rho_air

#Calculate the total mass (Accum mode)
totalMass = 0
for i in range(17,len(sizeBins),1):
    totalMass = totalMass + sizeBins[i]

#Convert to kg/kg
massMixingRatio_Accum = (totalMass * (10**6)) / rho_air

massMixingRatio_Total = massMixingRatio_Aitken + massMixingRatio_Accum


## 7e. Input results into dataframe (note: assumes constant distribution)

In [ ]:
for i in range(len(avg_4)):
    avg_4.loc[i,'Mass_Aitk'] = massMixingRatio_Aitken
    avg_4.loc[i,'Mass_Accu'] = massMixingRatio_Accum
    avg_4.loc[i,'Mass_Total'] = massMixingRatio_Total
    

## 7e. Save to spreadsheet

In [ ]:
avg_4.to_excel(spath + sname + '_avg.xlsx')


# 8. Plot profiles

In [ ]:
avg = pd.read_excel(spath + sname + '_avg.xlsx')

plt.figure(figsize=(10,14))

plt.subplot(521)
# Plot Temperature profile
plt.plot(avg['T_DI[K]'][:]-273.15,avg['z[m]'][:],label='T_DI',color='k')
plt.plot(avg['T_ND[K]'][:]-273.15,avg['z[m]'][:],label='T_ND',color='r')
plt.title('Average Temperature Profiles')
plt.xlabel('Temperature [C]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(522)
# Plot RH profile
plt.plot(avg['RH'][:],avg['z[m]'][:],label='RH',color='g')
plt.title('Average Relative Humidity Profile')
plt.xlabel('RH [%]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(523)
# Plot Potential Temperature profile
plt.plot(avg['theta_DI[K]'][:],avg['z[m]'][:],label='theta_DI',color='k')
plt.plot(avg['theta_ND[K]'][:],avg['z[m]'][:],label='theta_ND',color='r')
plt.title('Potential Temperature [K]')
plt.xlabel('Potential Temperature [K]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(524)
# Plot Vapour Mixing Ratio profile
plt.plot(avg['mix_ratio_DI[kg/kg]'][:],avg['z[m]'][:],label='mixing ratio DI',color='k')
plt.plot(avg['mix_ratio_ND[kg/kg]'][:],avg['z[m]'][:],label='mixing ratio ND',color='r')
plt.title('Vapour mixing ratio [kg/kg]')
plt.xlabel('Vapour Mixing Ratio [kg/kg]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(525)
# Plot Pressure profile
plt.plot(avg['P[hPa]'][:],avg['z[m]'][:],label='pressure',color='b')
plt.title('Average Pressure Profile')
plt.xlabel('Pressure [hPa]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(526)
# Plot u wind
plt.plot(avg['u[m/s]'][:],avg['z[m]'][:],label='u wind',color='k')
plt.title('Average U-wind Component')
plt.xlabel('U-wind [m/s]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(527)
# Plot Lat
plt.plot(avg['Lat'][:],avg['z[m]'][:],label='Latitude',color='grey')
plt.title('Average Latitude')
plt.xlabel('Latitude')
plt.ylabel('z[m]')
plt.legend()

plt.subplot(528)
# Plot v wind
plt.plot(avg['v[m/s]'][:],avg['z[m]'][:],label='v wind',color='k')
plt.title('Average V-wind Component')
plt.xlabel('V-wind [m/s]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(529)
# Plot Lon
plt.plot(avg['Lon'][:],avg['z[m]'][:],label='Longitude',color='grey')
plt.title('Average Longitude')
plt.xlabel('Longitude')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplot(5,2,10)
# Plot w wind
plt.plot(avg['w[m/s]'][:],avg['z[m]'][:],label='w wind',color='k')
plt.title('Average W-wind Component')
plt.xlabel('W-wind [m/s]')
plt.ylabel('Altitude [m]')
plt.legend()

plt.subplots_adjust(hspace=0.5)
plt.savefig(spath + sname + '_plots.')
plt.show()


# 9. Select the data from the specified input interval and save to new excel spreadsheet

In [ ]:
# Remove the extra column that was added during reindexing
avg = avg.drop(columns="Unnamed: 0")

# Select data for every interval_input m and save to a separate file
# Initialize empty array for use later
remove = []

# Loop through all the rows in avg
for i in range(len(avg)):
    # If the height[i] is not divisible by interval_input then add row i to list to be removed
    if avg['z[m]'][i]%interval_input > 0:
        remove.append(i)

# Remove the unwanted rows and then save the shortened dataset to an xlsx file
#avg.drop(remove).to_excel(spath + sname +'_input_values.xlsx')
avg = avg.drop(remove)
avg.to_excel(spath + sname + '_input_values.xlsx')

# 10. Plot initial profiles

In [ ]:
with PdfPages(spath + sname + '_plots.pdf') as pp:

    # Plot temperature profile
    plt.figure(figsize=(8,6))

    plt.plot(avg['T_DI[K]'][:]-273.15,avg['z[m]'][:],label='T_DI',color='r')
    plt.plot(avg['T_ND[K]'][:]-273.15,avg['z[m]'][:],label='T_ND',color='b',linestyle='--')
    plt.title('Average Temperature Profiles',fontsize=20)
    plt.xlabel('Temperature [C]',fontsize=14)
    plt.ylabel('Altitude [m]',fontsize=14)
    plt.legend()
    pp.savefig()
    plt.show()
    

    # zoomed in
    plt.figure(figsize=(8,6))

    plt.plot(avg['T_DI[K]'][:]-273.15,avg['z[m]'][:],label='T_DI',color='r')
    plt.plot(avg['T_ND[K]'][:]-273.15,avg['z[m]'][:],label='T_ND',color='b',linestyle='--')
    plt.title('Average Temperature Profiles',fontsize=20)
    plt.xlabel('Temperature [C]',fontsize=14)
    plt.ylabel('Altitude [m]',fontsize=14)
    plt.ylim([0,2000])
    plt.xlim([-20,0])
    plt.legend()
    pp.savefig()
    plt.show()


    # Plot Potential Temperature profile
    plt.figure(figsize=(8,6))

    plt.plot(avg['theta_DI[K]'][:],avg['z[m]'][:],label='theta_DI',color='r')
    plt.plot(avg['theta_ND[K]'][:],avg['z[m]'][:],label='theta_ND',color='b',linestyle='--')
    plt.title('Potential Temperature [K]',fontsize=20)
    plt.xlabel('Potential Temperature [K]',fontsize=14)
    plt.ylabel('Altitude [m]',fontsize=14)
    plt.legend()
    pp.savefig()
    plt.show()

    # zoomed in
    plt.figure(figsize=(8,6))

    plt.plot(avg['theta_DI[K]'][:],avg['z[m]'][:],label='theta_DI',color='r')
    plt.plot(avg['theta_ND[K]'][:],avg['z[m]'][:],label='theta_ND',color='b',linestyle='--')
    plt.title('Potential Temperature [K]',fontsize=20)
    plt.xlabel('Potential Temperature [K]',fontsize=14)
    plt.ylabel('Altitude [m]',fontsize=14)
    plt.ylim([0,2000])
    plt.xlim([260,285])
    plt.legend()
    pp.savefig()
    plt.show()

    # Plot Vapour Mixing Ratio profile
    plt.figure(figsize=(8,6))

    plt.plot(avg['mix_ratio_DI[kg/kg]'][:],avg['z[m]'][:],label='mixing ratio DI',color='r')
    plt.plot(avg['mix_ratio_ND[kg/kg]'][:],avg['z[m]'][:],label='mixing ratio ND',color='b',linestyle='--')
    plt.title('Vapour mixing ratio [kg/kg]',fontsize=20)
    plt.xlabel('Vapour Mixing Ratio [kg/kg]',fontsize=14)
    plt.ylabel('Altitude [m]',fontsize=14)
    plt.legend()
    pp.savefig()
    plt.show()

    # zoomed in
    plt.figure(figsize=(8,6))

    plt.plot(avg['mix_ratio_DI[kg/kg]'][:],avg['z[m]'][:],label='mixing ratio DI',color='r')
    plt.plot(avg['mix_ratio_ND[kg/kg]'][:],avg['z[m]'][:],label='mixing ratio ND',color='b',linestyle='--')
    plt.title('Vapour mixing ratio [kg/kg]',fontsize=20)
    plt.xlabel('Vapour Mixing Ratio [kg/kg]',fontsize=14)
    plt.ylabel('Altitude [m]',fontsize=14)
    plt.ylim([0,2000])
    plt.legend()
    pp.savefig()
    plt.show()





# 11. Convert profile data to format need by model and save to txt file

In [ ]:
# input_data = pd.read_excel(spath + sname + '_input_values.xlsx')

# file = open(spath + sname + '_input_data.txt','w')


# file.write('Initial Profile Data for MAC ' + sname +' \n\n\n')

# file.write('Altitude [m]:\n')
# file.write(str(input_data['z[m]'][:].round(0).tolist()))
# file.write('\n\n')

# file.write('Potential Temperature [K]:\n')
# file.write(str(input_data['theta_DI[K]'][:].round(2).tolist()))
# file.write('\n\n')

# file.write('Vapour Mixing Ratio [kg/kg]:\n')
# file.write(str(input_data['mix_ratio_DI[kg/kg]'][:].round(4).tolist()))
# file.write('\n\n')

# file.write('Aitken Aerosol Number Concentration [#/kg]:\n')
# x = np.format_float_scientific(input_data['NC_Aitk'][0], precision=2, exp_digits=2)
# for i in range(1,len(input_data),1):
#     x = x + ', ' + np.format_float_scientific(input_data['NC_Aitk'][i], precision=2, exp_digits=2)
# file.write(x)
# file.write('\n\n')

# file.write('Aitkne Aerosol Mass Mixing Ratio [kg/kg]:\n')
# x = np.format_float_scientific(input_data['Mass_Aitk'][0], precision=2, exp_digits=2)
# for i in range(1,len(input_data),1):
#     x = x + ', ' + np.format_float_scientific(input_data['Mass_Aitk'][i], precision=2, exp_digits=2)
# file.write(x)
# file.write('\n\n')

# file.write('Accumulation Aerosol Number Concentration [#/kg]:\n')
# x = np.format_float_scientific(input_data['NC_Accu'][0], precision=2, exp_digits=2)
# for i in range(1,len(input_data),1):
#     x = x + ', ' + np.format_float_scientific(input_data['NC_Accu'][i], precision=2, exp_digits=2)
# file.write(x)
# file.write('\n\n')

# file.write('Accumulation Aerosol Mass Mixing Ratio [kg/kg]:\n')
# x = np.format_float_scientific(input_data['Mass_Accu'][0], precision=2, exp_digits=2)
# for i in range(1,len(input_data),1):
#     x = x + ', ' + np.format_float_scientific(input_data['Mass_Accu'][i], precision=2, exp_digits=2)
# file.write(x)
# file.write('\n\n')

# file.write('Total Aerosol Number Concentration [#/kg]:\n')
# x = np.format_float_scientific(input_data['NC_Total'][0], precision=2, exp_digits=2)
# for i in range(1,len(input_data),1):
#     x = x + ', ' + np.format_float_scientific(input_data['NC_Total'][i], precision=2, exp_digits=2)
# file.write(x)
# file.write('\n\n')

# file.write('Total Aerosol Mass Mixing Ratio [kg/kg]:\n')
# x = np.format_float_scientific(input_data['Mass_Total'][0], precision=2, exp_digits=2)
# for i in range(1,len(input_data),1):
#     x = x + ', ' + np.format_float_scientific(input_data['Mass_Total'][i], precision=2, exp_digits=2)
# file.write(x)
# file.write('\n\n')

# file.write('U Wind [m/s]:\n')
# file.write(str(input_data['u[m/s]'][:].round(2).tolist()))
# file.write('\n\n')

# file.write('V Wind [m/s]:\n')
# file.write(str(input_data['v[m/s]'][:].round(2).tolist()))
# file.write('\n\n')

# file.close()



# 12. Create mcf file

In [ ]:
#Set aerosol type identifier for save name
#aeroType = '_accum'
#aeroType = '_aitken'
aeroType = '_accumAndAitken'

#Set variables for MONC configuration file
display_synopsis_frequency = 100
termination_time = 43260.
dtm = 0.2
xmlPath = "INSERT_FILEPATH"
modelSname = "INSERT_SNAME"
monsPerIO = 11
sampling_frequency = 20
sampling_frequency_3d = 200
mm = 600.0
mm1 = 60.0
diag_write_freq = 43200.0
walltime_limit = "04:00:00"

### surface data ###
thref = 273.31 #From calculations in <checkDataForProfileConstruction>
surface_pressure = 97457 #From calculations in <checkDataForProfileConstruction>
###              ###

x_size = 32
y_size = 32
z_size =37
dxx = 100
dyy = 100
zztop = 3950
kgd = "2,37"
hgd = "50,3950"
rmlmax = 23

### max cloud height ###
maxCloudHeight = 900 #Based on table in paper
###                  ###

z0 = "4.5e-4"
z0th = "2.2e-4"
fcoriol = 0.00014
dmptim = 0.001
zdmp = 3400.0
hdmp = 100.0
zSubs = "0.0,3950"
fSubs = "0.000005,0.000005"
lat = -75
lon = -32

### radiation time/date ###
rad_start_year = 2015
rad_start_day = 331
rad_start_time = 19
###                     ###
surface_albedo = 0.3

### Casim options ###
act = 4
inuc = 2
process_level = 0
aeroOption = 2
q_fields = 17
###               ###

#Format the profile data
z = str(input_data['z[m]'][1].round(0))
th = str(input_data['theta_DI[K]'][1].round(2))
u = str(input_data['u[m/s]'][1].round(2))
v = str(input_data['v[m/s]'][1].round(2))
q = str(input_data['mix_ratio_DI[kg/kg]'][1].round(4))
num_Aitk = np.format_float_scientific(input_data['NC_Aitk'][1], precision=2, exp_digits=2)
mass_Aitk = np.format_float_scientific(input_data['Mass_Aitk'][1], precision=2, exp_digits=2)
num_Accu = np.format_float_scientific(input_data['NC_Accu'][1], precision=2, exp_digits=2)
mass_Accu = np.format_float_scientific(input_data['Mass_Accu'][1], precision=2, exp_digits=2)
num_Total = np.format_float_scientific(input_data['NC_Total'][1], precision=2, exp_digits=2)
mass_Total = np.format_float_scientific(input_data['Mass_Total'][1], precision=2, exp_digits=2)

rand = str(input_data['RandomNoise'][1].round(4))

for i in range(2,len(input_data),1):
    z = z + ', ' + str(input_data['z[m]'][i].round(0))
    th = th + ', ' + str(input_data['theta_DI[K]'][i].round(2))
    u = u + ', ' + str(input_data['u[m/s]'][i].round(2))
    v = v + ', ' + str(input_data['v[m/s]'][i].round(2))
    q = q + ', ' + str(input_data['mix_ratio_DI[kg/kg]'][i].round(4))
    num_Aitk = num_Aitk + ', ' + np.format_float_scientific(input_data['NC_Aitk'][i], precision=2, exp_digits=2)
    mass_Aitk = mass_Aitk + ', ' + np.format_float_scientific(input_data['Mass_Aitk'][i], precision=2, exp_digits=2)
    num_Accu = num_Accu + ', ' + np.format_float_scientific(input_data['NC_Accu'][i], precision=2, exp_digits=2)
    mass_Accu = mass_Accu + ', ' + np.format_float_scientific(input_data['Mass_Accu'][i], precision=2, exp_digits=2)
    rand = rand + ', ' + str(input_data['RandomNoise'][i].round(4))
    
    
#Set which aerosol mode(s) to use
#names = "vapour, accum_sol_mass, accum_sol_number"
#mass = mass_Accu
#number = num_Accu
#names = "vapour, aitken_sol_mass, aitken_sol_number"
#mass = mass_Aitk
#number = num_Aitk
names = "vapour, accum_sol_mass, accum_sol_number, aitken_sol_mass, aitken_sol_number"


#Create MONC configuration file
with open(spath + sname + aeroType + '.mcf', "w") as file:

    file.write('# Global configuration\n')
    file.write('global_configuration=global_config\n\n')

    file.write('# Override global component defaults\n')
    file.write('simplesetup_enabled=.true.\n')
    file.write('smagorinsky_enabled=.true.\n')
    file.write('lower_bc_enabled=.true.\n')
    file.write('coriolis_enabled=.true.\n')
    file.write('damping_enabled=.true.\n\n')

    file.write('forcing_enabled=.false.\n\n')

    file.write('galilean_transformation=.false. # Needs debugging\n')
    file.write('randomnoise_enabled=.true.\n')
    file.write('mean_profiles_enabled=.true. #This must be set to true if running with damping\n')
    file.write('th_advection_enabled=.true.\n')

    file.write('iobridge_enabled=.true.\n')
    file.write('scalar_diagnostics_enabled=.true.\n')
    file.write('profile_diagnostics_enabled=.true.\n')
    file.write('diagnostics_3d_enabled=.true.\n\n')

    file.write('simplecloud_enabled=.false.\n')
    file.write('casim_enabled=.true.\n')
    file.write('socrates_couple_enabled=.false.\n\n')

    file.write('# Control configuration\n')
    file.write('display_synopsis_frequency=' + str(display_synopsis_frequency) + '\n')
    file.write('termination_time=' + str(termination_time) + '\n')
    file.write('dtm=' + str(dtm) + '\n\n')

    file.write('# IO server configuration\n')
    file.write('ioserver_configuration_file="' + xmlPath + '"\n')
    file.write('sname="' + modelSname + '"\n"')
    file.write('moncs_per_io_server=' + str(monsPerIO) + '\n')
    file.write('sampling_frequency=' + str(sampling_frequency) + '\n')
    file.write('3d_sampling_frequency=' + str(sampling_frequency_3d) + '\n')
    file.write('mm=' + str(mm) + '\n')
    file.write('mm1=' + str(mm1) + '\n')
    file.write('diag_write_freq=' + str(diag_write_freq) + '\n\n')

    file.write('# Checkpoint configuration\n')
    file.write('checkpoint_frequency=0\n')
    file.write('checkpoint_file="moncCheckpoint.nc"\n')
    file.write('checkpoint_walltime_frequency=10\n')
    file.write('walltime_limit=' + walltime_limit + '\n\n')

    file.write('# Advection choices\n')
    file.write('advection_flow_fields=pw\n')
    file.write('advection_theta_field=tvd\n')
    file.write('advection_q_fields=tvd\n\n')

    file.write('# CFL configuration\n')
    file.write('cfl_cvismax=0.2\n')
    file.write('cfl_cvelmax=0.2\n')
    file.write('cfl_dtmmax=2.0\n')
    file.write('cfl_dtmmin=0.001\n\n')

    file.write('# Simple setup configuration\n')
    file.write('thref0=' + str(thref) + '\n')
    file.write('surface_pressure=' + str(surface_pressure) + '\n')
    file.write('x_size=' + str(x_size) + '\n')
    file.write('y_size=' + str(y_size) + '\n')
    file.write('z_size=' + str(z_size) + '\n')
    file.write('dxx=' + str(dxx) + '\n')
    file.write('dyy=' + str(dyy) + '\n')
    file.write('zztop=' + str(zztop) + '\n')
    file.write('kgd=' + str(kgd) + '\n')
    file.write('hgd=' + str(hgd) + '\n')
    file.write('rmlmax=' + str(rmlmax) + '\n')
    file.write('enable_theta=.true.\n')
    file.write('use_anelastic_equations=.false.\n')
    file.write('passive_th=.false.\n')
    file.write('passive_q=.false.\n\n')

    file.write('\n')

    file.write('# Initialization of fields\n')
    file.write('l_init_pl_theta=.true.\n\n')
    file.write('z_init_pl_theta=' + str(z) + '\n')
    file.write('f_init_pl_theta=' + str(th) + '\n')
    file.write('l_init_pl_u=.true.\n')
    file.write('z_init_pl_u=' + str(z) + '\n')
    file.write('f_init_pl_u=' + str(u) + '\n')
    file.write('l_init_pl_v=.true.\n')
    file.write('z_init_pl_v=' + str(z) + '\n')
    file.write('f_init_pl_v=' + str(v) + '\n')
    file.write('l_init_pl_q=.true.\n')
    file.write('names_init_pl_q=' + str(names) + '\n')
    file.write('z_init_pl_q=' + str(z) + '\n')
    #file.write('f_init_pl_q=' + str(q) + ', ' + str(mass) + ', ' + str(number) + '\n\n\n')
    file.write('f_init_pl_q=' + str(q) + ', ' + str(mass_Accu) + ', ' + str(num_Accu) + str(mass_Aitk) + str(num_Aitk) + '\n\n\n')
    
    file.write('# Random noise\n')
    file.write('l_rand_pl_theta=.true.\n')
    file.write('z_rand_pl_theta=' + str(z) + '\n')
    file.write('f_rand_pl_theta=' + str(rand) + '\n\n')

    file.write('# Simple cloud\n')
    file.write('max_height_cloud=' + str(maxCloudHeight) + '\n\n')

    file.write('# Roughness lengths\n')
    file.write('z0=' + str(z0) + '\n')
    file.write('z0th=' + str(z0th) + '\n\n')

    file.write('# Coriolis\n')
    file.write('fcoriol=' + str(fcoriol) + '\n')
    file.write('surface_geostrophic_wind_x=5.0\n\n')

    file.write('# Damping configuration\n')
    file.write('dmptim=' + str(dmptim) + '\n')
    file.write('zdmp=' + str(zdmp) + '\n')
    file.write('hdmp=' + str(hdmp) + '\n\n')

    file.write('# Subsidence profile\n')
    file.write('l_subs_pl_theta=.true.\n')
    file.write('z_subs_pl=' + str(zSubs) + '\n')
    file.write('f_subs_pl=' + str(fSubs) + '\n')
    file.write('l_subs_pl_q=.true.\n\n')

    file.write('#SUBSIDENCE=1, DIVERGENCE=0\n')
    file.write('subsidence_input_type=0\n\n')

    file.write('# Socrates\n')
    file.write('i_cloud_representation = 1\n')
    file.write('latitude = ' + str(lat) + '\n')
    file.write('longitude = ' + str(lon) + '\n')
    file.write('rad_start_year = ' + str(rad_start_year) + '\n')
    file.write('rad_start_day = ' + str(rad_start_day) + '\n')
    file.write('rad_start_time = ' + str(rad_start_time) + '\n')
    file.write('rad_int_time = 0\n')
    file.write('surface_albedo = ' + str(surface_albedo) + '\n\n')

    file.write('mphys_nq_l=1\n')
    file.write('mphys_nd_l=1\n')
    file.write('mphys_nq_r=1\n')
    file.write('mphys_nq_i=1\n')
    file.write('mphys_nq_s=1\n')
    file.write('mphys_nq_g=1\n\n')

    file.write('# Casim options\n')
    file.write('option=22222\n')
    file.write('l_warm=.false.\n\n')

    file.write('iopt_act=' + str(act) + '\n')
    file.write('iopt_inuc=' + str(inuc) + '\n')
    file.write('process_level=' + str(process_level) + '\n')
    file.write('aerosol_option=' + str(aeroOption) + '\n')
    file.write('l_override_checks=.true.\n\n')

    file.write('number_q_fields=' + str(q_fields) + '\n')
    
